#Scalable scoring with Databricks pipeline

#### After training and selecting best model, it's now time to run inferencing on large datasets. This notebook introduce a method where you can setup a scoring pipeline to run your model against large dataset using Spark distributed processing framework

In [3]:
# import the Workspace class and check the azureml SDK version
# exist_ok checks if workspace exists or not.

from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
subscription_id = "be1b220b-fcad-4ce7-b323-6732ecc06c14" #you should be owner or contributor
resource_group = "pocdlkdevwu2rgtraning" #you should be owner or contributor
workspace_name = "commamlws01" #your workspace name
workspace_region = "westus2" #your region
ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      auth = InteractiveLoginAuthentication(force=True, tenant_id="be413eec-6262-4083-97c8-8c2a817c2fe1"),
                      resource_group = resource_group, 
                      location = workspace_region,
                      
                      exist_ok=True)



--------------------------------------------------------------------------- 
 ModuleNotFoundError Traceback (most recent call last)
 <command-2417002822658744> in <module> () 
 2 # exist_ok checks if workspace exists or not. 
 3 
 ----> 4 from azureml . core import Workspace
 5 from azureml . core . authentication import InteractiveLoginAuthentication
 6 subscription_id = "be1b220b-fcad-4ce7-b323-6732ecc06c14" #you should be owner or contributor 

 ModuleNotFoundError : No module named 'azureml'

## 1. Load spark mlib model and score

It's straight forward to score against a model trained and stored as spark ml

In [6]:
#Download the model from the best run to a local folder. Use this during inference. 

import os
from azureml.core.model import Model
import shutil
from pyspark.ml import Pipeline, PipelineModel

model_name = "crime_prediction_RDF.mml"
#spark ml can only load model from hdfs/dbfs file, not local file
model_name_dbfs_client_path = "/dbfs/mnt/models/"
model_name_dbfs_server_path = "dbfs:/mnt/models/"
#Initialize model and loading from Azure ML using latest version
model = Model(name = model_name,workspace = ws)
if os.path.isfile(model_name) or os.path.isdir(model_name):
    shutil.rmtree(model_name)

model.download(model_name_dbfs_client_path, exist_ok=True)
dbutils.fs.cp("file:"+model_name_dbfs_client_path,model_name_dbfs_server_path, True)

model_p_best = PipelineModel.load(model_name_dbfs_server_path+"/"+model_name)

In [7]:
#score the result
crime_df = spark.sql("select * from crime_dataset").na.drop()
result = model_p_best.transform(crime_df)
result.write.format("delta").mode("overwrite").saveAsTable("spark_mlprediction")
display(spark.sql("select * from spark_mlprediction"))

crime_count,year,week,day,district,primary_type,school_test_performance,population,Unemployment_Rte,Median_Household_Income,Average_Commute_Time,Area,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WT01,WT03,WT04,WT05,WT06,WT11,day_idx,district_idx,primary_type_idx,school_test_performance_idx,rawFeatures,features,prediction
5,2007,42,Wednesday,8.0,ASSAULT,Below Average,40950.4,5.799999999999583,58613.6,37.85999999998162,4.8,0.0,0.0,0.0,68.0,49.0,67.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,4.0,0.0,"List(0, 22, List(0, 1, 2, 3, 4, 5, 9, 10, 11, 18, 19, 20), List(42.0, 40950.4, 5.799999999999583, 58613.6, 37.85999999998162, 4.8, 68.0, 49.0, 67.0, 1.0, 8.0, 4.0))","List(0, 22, List(0, 1, 2, 3, 4, 5, 9, 10, 11, 18, 19, 20), List(42.0, 40950.4, 5.799999999999583, 58613.6, 37.85999999998162, 4.8, 68.0, 49.0, 67.0, 1.0, 8.0, 4.0))",3.30456799045883
7,2001,4,Friday,2.0,CRIMINAL DAMAGE,Below Average,38408.71428571428,6.557142857140475,34773.57142857143,30.58571428572399,2.2857142857142856,0.0,0.0,3.0,26.0,13.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,3.0,0.0,"List(0, 22, List(0, 1, 2, 3, 4, 5, 8, 9, 10, 11, 19, 20), List(4.0, 38408.71428571428, 6.557142857140475, 34773.57142857143, 30.58571428572399, 2.2857142857142856, 3.0, 26.0, 13.0, 23.0, 16.0, 3.0))","List(0, 22, List(0, 1, 2, 3, 4, 5, 8, 9, 10, 11, 19, 20), List(4.0, 38408.71428571428, 6.557142857140475, 34773.57142857143, 30.58571428572399, 2.2857142857142856, 3.0, 26.0, 13.0, 23.0, 16.0, 3.0))",5.425990224617209
6,2007,37,Wednesday,8.0,OTHER OFFENSE,Below Average,89597.0,5.7566666666668285,55065.0,36.799999999970254,7.0,0.0,0.0,0.0,72.0,46.0,51.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,0.0,0.0,"List(0, 22, List(0, 1, 2, 3, 4, 5, 9, 10, 11, 18, 19), List(37.0, 89597.0, 5.7566666666668285, 55065.0, 36.799999999970254, 7.0, 72.0, 46.0, 51.0, 1.0, 8.0))","List(0, 22, List(0, 1, 2, 3, 4, 5, 9, 10, 11, 18, 19), List(37.0, 89597.0, 5.7566666666668285, 55065.0, 36.799999999970254, 7.0, 72.0, 46.0, 51.0, 1.0, 8.0))",3.927383520052744
7,2007,37,Sunday,8.0,BATTERY,Below Average,93071.14285714286,5.758571428571632,54355.57142857143,36.81428571425469,7.142857142857143,0.0,0.0,0.0,63.0,46.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,8.0,2.0,0.0,"List(0, 22, List(0, 1, 2, 3, 4, 5, 9, 10, 11, 18, 19, 20), List(37.0, 93071.14285714286, 5.758571428571632, 54355.57142857143, 36.81428571425469, 7.142857142857143, 63.0, 46.0, 52.0, 6.0, 8.0, 2.0))","List(0, 22, List(0, 1, 2, 3, 4, 5, 9, 10, 11, 18, 19, 20), List(37.0, 93071.14285714286, 5.758571428571632, 54355.57142857143, 36.81428571425469, 7.142857142857143, 63.0, 46.0, 52.0, 6.0, 8.0, 2.0))",10.815554341368443
9,2018,6,Sunday,11.0,NARCOTICS,Below Average,37590.22222222222,5.351111111110212,29055.666666666668,35.46666666665591,4.0,0.5500000000000055,3.0,14.0,23.0,16.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0,5.0,0.0,"List(1, 22, List(), List(6.0, 37590.22222222222, 5.351111111110212, 29055.666666666668, 35.46666666665591, 4.0, 0.5500000000000055, 3.0, 14.0, 23.0, 16.0, 21.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 6.0, 5.0, 0.0))","List(1, 22, List(), List(6.0, 37590.22222222222, 5.351111111110212, 29055.666666666668, 35.46666666665591, 4.0, 0.5500000000000055, 3.0, 14.0, 23.0, 16.0, 21.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 6.0, 5.0, 0.0))",12.590593211120638
4,2002,26,Monday,11.0,CRIMINAL DAMAGE,Poor,64267.0,5.2100000000019255,41711.0,35.80000000001299,4.0,0.0,0.0,0.0,92.0,64.0,74.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,6.0,3.0,1.0,"List(0, 22, List(0, 1, 2, 3, 4, 5, 9, 10, 11, 18, 19, 20, 21), List(26.0, 64267.0, 5.2100000000019255, 41711.0, 35.80000000001299, 4.0, 92.0, 64.0, 74.0, 4.0, 6.0, 3.0, 1.0))","List(0, 22, List(0, 1, 2, 3, 4, 5, 9, 10, 11, 18, 19, 20, 21), List(26.0, 64267.0, 5.2100000000019255, 41711.0, 35.80000000001299, 4.0, 92.0, 64.0, 74.0, 4.0, 6.0, 3.0, 1.0))",1.9868581290841538
9,2015,3,Saturday,11.0,BATTERY,Below Average,38105.0,5.3299999999993695,28695.0,36.19999999998712,4.0,0.0,0.0,0.0,37.0,21.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,6.0,2.0,0.0,"List(0, 22, List(0, 1, 2, 3, 4, 5, 9, 10, 11,

## 2. Load SKLearn model (or ternsorflow etc..) and score

## If you trained your model with SKLearn or Tensorflow (not using Pyspark), it's still possible to run scalable scoring against large dataset. The approach below uses Pandas UDF to score batches of data from loaded SKLearn model you trained in prediction notebook

In [10]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
import pandas as pd
from sklearn.externals import joblib


def make_predictions(sc, df, feature_cols, model_path, tf_path):
    """
    Make predictions.
    
    Arguments:
        sc: SparkContext.
        df (pyspark.sql.DataFrame): Input data frame containing feature_cols.
        feature_cols (list[str]): List of feature columns in df.
        model_path (str): Path to model on Spark driver

    Returns:
        df (pyspark.sql.DataFrame): Output data frame with probability column.
    """
    # Load classifier and broadcast to executors.
    model = sc.broadcast(joblib.load(model_path))
    tf = sc.broadcast(joblib.load(tf_path))

    # Define Pandas UDF
    @F.pandas_udf(returnType=DoubleType(), functionType=F.PandasUDFType.SCALAR)
    def predict(*cols):
        # Columns are passed as a tuple of Pandas Series'.
        # Combine into a Pandas DataFrame
        pdCrime = pd.concat(cols, axis=1)
        pdCrime.columns = feature_cols
        X = tf.value.transform(pdCrime)

#         # Make predictions .
        predictions = model.value.predict(X)
        # Return Pandas Series of predictions.
        return pd.Series(predictions)

    # Make predictions on Spark DataFrame.
    df = df.withColumn("predictions", predict(*feature_cols))
    
    return df

In [11]:
#Download the model from the best run to a local folder. Use this during inference. 

import os
from azureml.core.model import Model
import shutil
from sklearn.externals import joblib

from pyspark.ml import Pipeline, PipelineModel
#Initialize model and loading from Azure ML using latest version

model_name = "ml.joblib"
#spark ml can only load model from hdfs/dbfs file, not local file
model_name_dbfs_client_path = "/dbfs/mnt/models/"+model_name

ft_file = 'ft.joblib'
ml_file = 'ml.joblib'

ft_path = os.path.join(model_name_dbfs_client_path+"/"+model_name,ft_file )
ml_path = os.path.join(model_name_dbfs_client_path+"/"+model_name,ml_file )

model = Model(name = model_name,workspace = ws)
if os.path.isfile(model_name) or os.path.isdir(model_name):
    shutil.rmtree(model_name)

model.download(model_name_dbfs_client_path, exist_ok=True)
crime_df = spark.sql("select * from crime_dataset").na.drop()
feature_cols = ['week', 'day', 'district', 'primary_type', 'school_test_performance',
       'population', 'Unemployment_Rte', 'Median_Household_Income',
       'Average_Commute_Time', 'Area', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN',
       'TOBS', 'WT01', 'WT03', 'WT04', 'WT05', 'WT06', 'WT11']
sc = spark.sparkContext

pred=  make_predictions(sc, crime_df, feature_cols, ml_path, ft_path)
pred.write.format("delta").mode("overwrite").saveAsTable("sk_learn_prediction")
display(spark.sql("select * from sk_learn_prediction"))

crime_count,year,week,day,district,primary_type,school_test_performance,population,Unemployment_Rte,Median_Household_Income,Average_Commute_Time,Area,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WT01,WT03,WT04,WT05,WT06,WT11,predictions
2,2005,26,Friday,8.0,OTHER OFFENSE,Below Average,102621.0,5.659999999999002,48455.0,35.34999999999184,8.0,0.0,0.0,0.0,78.0,56.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0473694455379046
15,2006,15,Monday,8.0,THEFT,Below Average,83215.73333333334,5.832666666666565,47327.8,38.25333333330069,6.533333333333333,0.0,0.0,0.0,57.0,34.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,6.035161963304329
3,2010,3,Sunday,8.0,CRIMINAL DAMAGE,Average,55026.0,5.3800000000004955,56616.0,33.0,11.0,0.41000000000000386,0.0,0.0,46.0,35.0,46.0,1.0,0.0,0.0,0.0,0.0,0.0,1.7734570676778214
3,2010,4,Saturday,8.0,OTHER OFFENSE,Below Average,65278.0,5.7433333333332115,60031.0,36.69999999997923,6.0,0.0,0.0,1.0,21.0,4.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2179757050982114
3,2002,5,Friday,11.0,THEFT,Poor,64267.0,5.2100000000019255,41711.0,35.80000000001299,4.0,0.13999999999999632,0.7999999999999808,6.0,36.0,27.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2821254176523045
3,2002,10,Thursday,11.0,THEFT,Poor,53854.333333333336,5.016666666668407,37402.333333333336,36.666666666701296,4.0,0.0,0.0,7.0,46.0,33.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2821254176523045
6,2005,4,Thursday,8.0,CRIMINAL DAMAGE,Below Average,73665.33333333333,5.771666666666197,51680.5,37.283333333312775,6.333333333333333,0.09999999999999838,1.5,6.0,30.0,8.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,5.995705092793924
1,2005,25,Friday,8.0,OTHER OFFENSE,Below Average,113916.0,5.770000000000446,50099.0,36.89999999996128,8.0,0.0,0.0,0.0,90.0,70.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9603017636585631
1,2005,25,Saturday,8.0,CRIMINAL DAMAGE,Average,55026.0,5.380000000000495,56616.0,33.0,11.0,0.0,0.0,0.0,95.0,71.0,81.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7734570676778214
1,2007,45,Monday,8.0,OTHER OFFENSE,Average,55026.0,5.380000000000495,56616.0,33.0,11.0,0.0,0.0,0.0,53.0,40.0,47.0,0.0,0.0,0.0,0.0,0.0,0.0,1.3111282249108864


## 3. Scoring with AKS deployed services

In [13]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
import pandas as pd
import json
import requests
import ast
import time
def make_predictions_aks(sc, df, feature_cols):
    """
    Make predictions.
    
    Arguments:
        sc: SparkContext.
        df (pyspark.sql.DataFrame): Input data frame containing feature_cols.
        feature_cols (list[str]): List of feature columns in df.

    Returns:
        df (pyspark.sql.DataFrame): Output data frame with probability column.
    """
    scoring_uri ='http://13.66.160.122/api/v1/service/crime-pred-service-1/score'
    api_key ='i4J6SsvDPI3XuifhtI9NIhxrYGtt2Caz'
    # Define Pandas UDF
    @F.pandas_udf(returnType=DoubleType(), functionType=F.PandasUDFType.SCALAR)
    def predict(*cols):
        # Columns are passed as a tuple of Pandas Series'.
        # Combine into a Pandas DataFrame
        pdCrime = pd.concat(cols, axis=1)
        pdCrime.columns = feature_cols
        json_raw = json.dumps(pdCrime.to_json(orient='split'))
        headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 
        
        
        response = requests.post(scoring_uri, data=json_raw, headers=headers)
        while response.status_code != 200:
          time.sleep(1)
          response = requests.post(scoring_uri, data=json_raw, headers=headers)


#         # Make predictions .
        predictions_json = json.loads(response.content.decode("utf-8"))
        pred_list =ast.literal_eval(predictions_json)
        predictions =pd.Series(pred_list)
        # Return Pandas Series of predictions.
        return predictions

    # Make predictions on Spark DataFrame.
    df = df.withColumn("predictions", predict(*feature_cols))
    
    return df

In [14]:

crime_df = spark.sql("select * from crime_dataset").na.drop()
feature_cols = ['week', 'day', 'district', 'primary_type', 'school_test_performance',
       'population', 'Unemployment_Rte', 'Median_Household_Income',
       'Average_Commute_Time', 'Area', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN',
       'TOBS', 'WT01', 'WT03', 'WT04', 'WT05', 'WT06', 'WT11']
sc = spark.sparkContext

pred=  make_predictions_aks(sc, crime_df, feature_cols)
pred.write.format("delta").mode("overwrite").saveAsTable("aks_prediction")
display(spark.sql("select * from aks_prediction"))
# display(pred)

crime_count,year,week,day,district,primary_type,school_test_performance,population,Unemployment_Rte,Median_Household_Income,Average_Commute_Time,Area,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WT01,WT03,WT04,WT05,WT06,WT11,predictions
3,2002,35,Tuesday,11.0,OTHER OFFENSE,Below Average,35016.333333333336,5.456666666664417,30859.0,31.79999999999983,4.0,0.0,0.0,0.0,83.0,64.0,68.0,0.0,0.0,0.0,0.0,0.0,0.0,3.3795947497664534
1,2002,36,Friday,11.0,OTHER OFFENSE,Poor,64267.0,5.2100000000019255,41711.0,35.80000000001299,4.0,0.0,0.0,0.0,84.0,56.0,64.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1576076368590478
5,2002,30,Sunday,11.0,CRIMINAL DAMAGE,Below Average,35325.2,5.443999999997912,30642.6,32.23999999999856,4.0,0.07999999999999854,0.0,0.0,89.0,71.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,4.463335955091248
11,2007,29,Tuesday,7.0,THEFT,Below Average,40916.0,6.079999999999537,33080.0,42.89999999995529,4.0,0.12000000000000223,0.0,0.0,83.0,68.0,69.0,0.0,0.0,0.0,0.0,0.0,0.0,5.079417299285671
4,2007,18,Friday,7.0,OTHER OFFENSE,Below Average,40916.0,6.079999999999536,33080.0,42.89999999995529,4.0,0.39999999999999,0.0,0.0,73.0,50.0,53.0,0.0,0.0,0.0,0.0,0.0,0.0,1.613146631119332
4,2007,30,Sunday,7.0,THEFT,Below Average,40916.0,6.079999999999536,33080.0,42.89999999995529,4.0,0.0,0.0,0.0,81.0,64.0,73.0,0.0,0.0,0.0,0.0,0.0,0.0,5.079417299285671
4,2007,31,Tuesday,7.0,THEFT,Poor,35921.0,6.6199999999964385,23875.0,40.69999999998456,4.0,0.0,0.0,0.0,87.0,65.0,73.0,0.0,0.0,0.0,0.0,0.0,0.0,1.8106751579276663
6,2007,18,Tuesday,8.0,OTHER OFFENSE,Below Average,77437.5,5.75000000000002,57548.0,36.74999999997474,6.5,0.0,0.0,0.0,47.0,33.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,2.434797070777485
2,2007,20,Tuesday,8.0,OTHER OFFENSE,Below Average,113916.0,5.770000000000446,50099.0,36.89999999996128,8.0,0.1699999999999954,0.0,0.0,75.0,55.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9003235665230092
3,2006,1,Friday,25.0,OTHER OFFENSE,Poor,81693.66666666667,5.196666666668569,47852.333333333336,35.53333333332731,4.666666666666667,0.0,0.0,0.0,36.0,28.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,3.3295801826600884


#Write result to SQL DW Table

In [16]:
#Using High PErformance Databricks SQL DW driver. STep 1 is to setup spark conf with intermediary blob credentials
storage_account = "i360pocstorage"
container= "sqldwdatabricks"
spark.conf.set(
  "fs.azure.account.key."+storage_account+".blob.core.windows.net",
  "Ql0T2f5vdg3nYtxDLb68BbA777B9cChFaH766Ufg6oztDtgvvoWtbRV2fP8wpGR8Z1KduRvqVwimKkU5pJSi1A==")
spark.conf.set("spark.network.timeout","10000000")
#Step 2: fill in SQL DW credentials, in real development, use secret scope insftead
username = 'srvadmin'
password = 'sqlserver@2019'
servername = 'jamestestsqlserver'
database = 'jamessqldw'

tempdir = "wasbs://"+container+"@"+storage_account+".blob.core.windows.net/"
connection_string = "jdbc:sqlserver://{0}.database.windows.net:1433;database={1};user={2}@{3};password={4}".format(servername, database,username,servername,password,servername)


In [17]:
#Read data from the table you stored prediction result in previous step into a dataframe
sql_sk_pred_df = spark.sql("select * from sk_learn_prediction")

In [18]:
#Write out to a table in SQL DW
sql_sk_pred_df.write \
  .format("com.databricks.spark.sqldw") \
  .option("url", connection_string) \
  .option("forwardSparkAzureStorageCredentials", "true") \
  .option("dbTable", "dbo.sk_learn_prediction") \
  .option("tempDir", tempdir) \
  .mode("overwrite") \
  .save()

In [19]:
#You can also read from SQL DW

pred_out = spark.read \
  .format("com.databricks.spark.sqldw") \
  .option("url", connection_string) \
  .option("tempDir",tempdir ) \
  .option("forwardSparkAzureStorageCredentials", "true") \
  .option("dbTable", "dbo.sk_learn_prediction") \
  .load()
display(pred_out)

crime_count,year,week,day,district,primary_type,school_test_performance,population,Unemployment_Rte,Median_Household_Income,Average_Commute_Time,Area,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WT01,WT03,WT04,WT05,WT06,WT11,predictions
1,2003,12,Tuesday,20.0,OTHER OFFENSE,Average,38104.0,4.790000000000745,55851.0,31.59999999999546,3.0,0.0,0.0,0.0,72.0,39.0,42.0,1.0,0.0,0.0,0.0,0.0,0.0,1.3039844862487096
4,2007,39,Wednesday,20.0,THEFT,Below Average,60701.5,4.894999999998316,49228.5,35.749999999996035,2.5,0.8099999999999757,0.0,0.0,83.0,55.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,4.654262387476292
2,2007,39,Saturday,20.0,OTHER OFFENSE,Below Average,72220.5,4.979999999998771,48849.0,35.34999999998782,3.5,0.0,0.0,0.0,74.0,51.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9812549747165524
1,2014,2,Friday,20.0,CRIMINAL DAMAGE,Below Average,65790.0,5.009999999999427,44410.0,35.09999999999049,3.0,0.029999999999999933,0.0,8.0,31.0,7.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,2.022262617570295
4,2006,4,Tuesday,14.0,OTHER OFFENSE,Below Average,78605.25,5.394999999997717,48515.0,30.69999999997672,3.75,0.0,0.0,1.0,61.0,20.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0473694455379046
4,2008,52,Wednesday,16.0,OTHER OFFENSE,Average,64195.5,5.004999999998152,61383.5,34.349999999996044,6.0,0.44999999999999374,0.0,0.0,37.0,18.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,1.6189793654356819
2,2009,7,Thursday,16.0,OTHER OFFENSE,Average,74298.0,5.089999999997645,62214.0,36.199999999992095,7.0,0.9099999999999916,0.0,0.0,54.0,22.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4744679089798245
3,2011,32,Monday,14.0,OTHER OFFENSE,Below Average,87291.0,5.309999999998083,46572.0,30.699999999974906,4.0,0.10999999999999949,0.0,0.0,88.0,68.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,1.3915293695633921
1,2011,31,Thursday,14.0,THEFT,Average,92084.0,5.100000000003252,55303.0,32.09999999999543,5.0,0.0,0.0,0.0,85.0,65.0,69.0,0.0,0.0,0.0,0.0,0.0,0.0,3.1689734605476723
1,2011,32,Thursday,14.0,OTHER OFFENSE,Below Average,52548.0,5.649999999996617,54344.0,30.69999999998215,3.0,0.0,0.0,0.0,78.0,57.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,1.6373666513183196
